In [3]:
import os
import re
import sys
import config
import kmeans
import constants
from sklearn import cluster
import numpy as np
import pandas as pd

import plotly.plotly as py

import extract
import transform
import load

from scipy.stats import zscore

#Pull the main data frame
d_ci = extract.extract_all_lazy()

# Register transform functions here, create them in the transform.py
column_operations = {'ANSWER_num_rooms':transform.answer_num_rooms,
                     'ANSWER_ann_op_rev':transform.answer_ann_op_rev,
                     'ANSWER_ann_revenue':transform.answer_ann_revenue,
                     'ANSWER_num_employees_pos':transform.answer_num_employees_pos,
                     'create_Executive':transform.create_Executive,
                     'ANSWER_cur_base_pay':transform.answer_cur_base_pay
                     #'create_RevPAR':transform.create_RevPAR
                    }

In [4]:
#Create the Modeling dataset
clean_data  = transform.clean_model(d_ci, column_operations)
d_ci_v2 = clean_data['data']
metadata = clean_data['metadata']



In [6]:
metadata

['create_Executive_PRIOR :: Rows: 74732, Columns: 50',
 'create_Executive_POST :: Rows: 74732, Columns: 51',
 'ANSWER_ann_revenue_PRIOR :: Rows: 74732, Columns: 51',
 'ANSWER_ann_revenue_POST :: Rows: 24994, Columns: 51',
 'ANSWER_num_employees_pos_PRIOR :: Rows: 24994, Columns: 51',
 'ANSWER_num_employees_pos_POST :: Rows: 24993, Columns: 51',
 'ANSWER_ann_op_rev_PRIOR :: Rows: 24993, Columns: 51',
 'ANSWER_ann_op_rev_POST :: Rows: 19215, Columns: 51',
 'ANSWER_cur_base_pay_PRIOR :: Rows: 19215, Columns: 51',
 'ANSWER_cur_base_pay_POST :: Rows: 19215, Columns: 53',
 'ANSWER_num_rooms_PRIOR :: Rows: 19215, Columns: 53',
 'ANSWER_num_rooms_POST :: Rows: 18617, Columns: 53']

In [2]:

#Preprocessing step before modeling. Normalize numeric score via Z-score
features = ['CREATED_cur_base_pay_hrs','ANSWER_num_rooms','ANSWER_num_employees_pos']
features = features + ['Gaming/Casino', #Service Types
                                      'Golf/Country Club',
                                      'Lodging - Full Service',
                                      'Lodging - Select Service',
                                      'Vacation Ownership']
features_norm = [f + "_" for f in features]
d_ci_v2[features_norm] = d_ci_v2[features].apply(lambda x: pd.to_numeric(x), axis=1).apply(zscore)



#markets/cities go in 0 position
dimensions = ['CITYMARKET','POSITION']
modeling = d_ci_v2[features_norm + dimensions]

# The top X number of markets by the number of properties
number_of_markets = 50
number_of_clusters = 3
markets = transform.top_markets_by_property(modeling, number_of_markets, dimensions[0])
positions = modeling['POSITION'].unique().tolist()

transform.py:20: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [2]:
positions

NameError: name 'positions' is not defined

In [3]:
total_frame = pd.DataFrame()
distance_frame = pd.DataFrame()

for market in markets:
        temp_frame = modeling[modeling['CITYMARKET'] == market]
        temp_frame = transform.run_model(temp_frame, features_norm, number_of_clusters)
        total_frame = total_frame.append(temp_frame)
    #for job in positions:
#         temp_frame_v1 = temp_frame[temp_frame['POSITION'] == job]
#         if temp_frame_v1.shape[0] > number_of_clusters:
#             print market, " ", job, " ",temp_frame_v1.shape
#             temp_frame_v1 = transform.run_model(temp_frame_v1, features_norm, number_of_clusters)
#             total_frame = total_frame.append(temp_frame_v1)
t_frame = total_frame.merge(d_ci_v2)

transform.py:120: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

transform.py:121: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [4]:
t_frame.to_csv("ww_top_30_markets_out.csv", index=False, encoding='utf8')

In [5]:
kwik_frame = load.kwik_analytics(t_frame, False, markets)

## Below will have to be moved to a more permanent home. Script to create Dash_v1 data

In [6]:
hotels = kwik_frame['hotels']
jobs = kwik_frame['jobs']

hotels.to_csv(os.path.join(os.getcwd(),'..','..','data','processed','ww_hotels_30.csv'))
jobs.to_csv(os.path.join(os.getcwd(),'..','..','data','processed','ww_jobs_30.csv'))


In [15]:
test = jobs[jobs['CITYMARKET'] == 'Atlanta']
test_v2 = test[test['POSITION'] == 'Bus Person']
test_v2 = test_v2[test_v2['clusters'] == 0]
#values = ['clusters','min','25','50','75','max']

test_v3 = test_v2[values]
test_v3



,min,25,50,75,max
135,4.5,9.135,9.78,10.695,10.9


In [13]:
data_butt=[]
for cluster in test_v3['clusters']:
    temp_dict = {}
    test_v4 = test_v3[test_v3['clusters'] == cluster]
    values = ['min','25','50','75','max']
    temp_dict['x'] = values
    temp_dict['y'] =list(test_v4[values].values[0])
    temp_dict['name'] = "Tier: {}".format(cluster)
    temp_dict['type'] = ['scatter']
    data_butt.append(temp_dict)

KeyError: 'clusters'

In [9]:
d = pd.Series(jobs['CITYMARKET'].unique()).to_dict()
payload = []
for key in d.keys():
    newdict = {}
    newdict['label'] = d[key]
    newdict['value'] = d[key]
    payload.append(newdict)

In [10]:
generate_mselect_data(hotels,'PROPERTY_NAME')

NameError: name 'generate_mselect_data' is not defined

In [ ]:
generate_mselect_data(jobs, 'POSITION')

##Visualization and Cluster Analysis##